# Autoscaling

Use the Python SDK to [get started with auto scaling](http://docs.aws.amazon.com/autoscaling/latest/userguide/GettingStartedTutorial.html)

## VPC Context

In [ ]:
import boto3

vpc_client = boto3.client('ec2')

In [ ]:
# Note that is you run this in an older account that supports EC2-Classic there might not be 
# a default VPC
response = vpc_client.describe_vpcs()
vpcs = response['Vpcs']
print vpcs

def_vpc = ''
for vpc in vpcs:
    if vpc['IsDefault'] == True:
        def_vpc = vpc['VpcId']
        break
        
print 'default vpc is {}'.format(def_vpc)

In [ ]:
response = vpc_client.describe_subnets(
    Filters=[
        {
            'Name':'vpc-id',
            'Values': [
                def_vpc
            ]
        }
    ]
)

print response

subnets = response['Subnets']

subnet_ids = []

for sn in subnets:
    subnet_ids.append(sn['SubnetId'])
    
print subnet_ids

subnet_list = ",".join(subnet_ids)
print subnet_list

## Create a Launch Config

In [ ]:

client = boto3.client('ec2')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
as_client = boto3.client('autoscaling')

In [ ]:
response = as_client.create_launch_configuration(
    LaunchConfigurationName='MyLaunchConfig',
    ImageId=ami_id,
    InstanceType='t2.micro'
)

print response

In [ ]:
response = as_client.describe_launch_configurations(
    LaunchConfigurationNames=[
        'MyLaunchConfig',
    ]
)

print response

In [ ]:
ec2_client = boto3.client('ec2')

In [ ]:
#
# TODO - modify this to create the instances in a VPC by using the 
# vpc subnets above in the asg instead of availability zone
# attr is VPCZoneIdentifier and the value is a comma separated list
# of subnet identifiers
#

response = ec2_client.describe_availability_zones()

zone_descriptions = response['AvailabilityZones']
print zone_descriptions

zones = []
for zd in zone_descriptions:
    zones.append(zd['ZoneName'])
    
print zones

## Create an AutoScaling Group

In [ ]:
response = as_client.create_auto_scaling_group(
    AutoScalingGroupName='MyASG',
    LaunchConfigurationName='MyLaunchConfig',
    DesiredCapacity=1,
    MinSize=1,
    MaxSize=2,
    VPCZoneIdentifier=subnet_list
    )

print response

In [ ]:
response = as_client.describe_auto_scaling_groups(
    AutoScalingGroupNames=[
        'MyASG',
    ]
    )

print response

## Clean up

In [ ]:
response = as_client.delete_auto_scaling_group(
    AutoScalingGroupName='MyASG',
    ForceDelete=True
)

print response

In [ ]:
response = as_client.delete_launch_configuration(
    LaunchConfigurationName='MyLaunchConfig'
)

print response